In [1]:

function is_word_in_language(regex::Regex, word::AbstractString)::Bool
    
    regex_str = string(regex)

    

    
    if startswith(regex_str, "r\"^")
        regex_str = regex_str[4:end-1] 
    elseif startswith(regex_str, "r\"")
        regex_str = regex_str[3:end-1] 
    else
        
        regex_str = regex_str
    end

    
    if endswith(regex_str, "\$")
        regex_str = regex_str[1:end-2]
    end

    
    

    
    full_match_regex = Regex("^" * regex_str * "\$")

    return match(full_match_regex, word) !== nothing
end


R1 = r"(a|b)(a|b)"

R2 = r"01*"

R3 = r"\d+"
is_word_in_language(Regex("(aabaa|aaaba|abaaaa)*(a|b)a(a|b)(ab|aab)*"),"abb")

false

In [2]:



struct DFA{T}
    states::Set{T}
    start_state::T
    final_states::Set{T}
    transitions::Dict{Tuple{T, Char}, T}
end


struct NFA{T}
    states::Set{T}
    start_state::T
    final_states::Set{T}
    transitions::Dict{Tuple{T, Union{Char, Nothing}}, Set{T}}
end



function parse_states(s::AbstractString)::Set{Int}
    if isempty(strip(s))
        return Set{Int}()
    end
    return Set(parse(Int, strip(x)) for x in split(s, ','))
end


function parse_dfa(states_str::AbstractString, start_str::AbstractString,
                   final_str::AbstractString, transitions_str::AbstractString)::DFA{Int}

    states = parse_states(states_str)
    start_state = parse(Int, strip(start_str))
    final_states = parse_states(final_str)
    transitions = Dict{Tuple{Int, Char}, Int}()

    for transition_entry in split(transitions_str, ';', keepempty=false)
        parts = split(transition_entry, ',', limit=3)
        length(parts) != 3 && throw(ErrorException("Неверный формат перехода DFA: $transition_entry"))

        from_state = parse(Int, strip(parts[1]))
        symbol_str = strip(parts[2])
        to_state = parse(Int, strip(parts[3]))

        length(symbol_str) != 1 && throw(ErrorException("Символ должен быть одиночным: $symbol_str"))
        symbol = first(symbol_str)

        transitions[(from_state, symbol)] = to_state
    end

    return DFA(states, start_state, final_states, transitions)
end


function parse_nfa(states_str::AbstractString, start_str::AbstractString,
                   final_str::AbstractString, transitions_str::AbstractString)::NFA{Int}

    states = parse_states(states_str)
    start_state = parse(Int, strip(start_str))
    final_states = parse_states(final_str)
    transitions = Dict{Tuple{Int, Union{Char, Nothing}}, Set{Int}}()

    for transition_entry in split(transitions_str, ';', keepempty=false)
        parts = split(transition_entry, ',', limit=3)
        length(parts) != 3 && throw(ErrorException("Неверный формат перехода NFA: $transition_entry"))

        from_state = parse(Int, strip(parts[1]))
        symbol_str = strip(parts[2])
        to_states = parse_states(parts[3])

        
        local symbol_key::Union{Char, Nothing}
        if symbol_str == "ε" || symbol_str == "E"
            symbol_key = nothing
        elseif length(symbol_str) == 1
            symbol_key = first(symbol_str)
        else
            throw(ErrorException("Неверный символ в NFA переходе: $symbol_str"))
        end

        key = (from_state, symbol_key)

        
        if haskey(transitions, key)
            union!(transitions[key], to_states)
        else
            transitions[key] = to_states
        end
    end

    return NFA(states, start_state, final_states, transitions)
end



function check_dfa(dfa::DFA{T}, word::AbstractString)::Bool where T
    current_state = dfa.start_state
    for char in word
        transition_key = (current_state, char)
        if haskey(dfa.transitions, transition_key)
            current_state = dfa.transitions[transition_key]
        else
            return false
        end
    end
    return current_state in dfa.final_states
end


function epsilon_closure(nfa::NFA{T}, states::Set{T})::Set{T} where T
    stack = collect(states)
    closure = copy(states)

    while !isempty(stack)
        q = pop!(stack)
        epsilon_key = (q, nothing)

        if haskey(nfa.transitions, epsilon_key)
            for next_q in nfa.transitions[epsilon_key]
                if next_q ∉ closure
                    push!(closure, next_q)
                    push!(stack, next_q)
                end
            end
        end
    end
    return closure
end

function check_nfa(nfa::NFA{T}, word::AbstractString)::Bool where T
    current_states = epsilon_closure(nfa, Set([nfa.start_state]))

    for char in word
        next_states_prime = Set{T}()
        for q in current_states
            transition_key = (q, char)
            if haskey(nfa.transitions, transition_key)
                union!(next_states_prime, nfa.transitions[transition_key])
            end
        end
        current_states = epsilon_closure(nfa, next_states_prime)
        if isempty(current_states)
            return false
        end
    end

    return !isempty(intersect(current_states, nfa.final_states))
end



println("=========================================")
println("       ДЕМОНСТРАЦИЯ (DFA И NFA)          ")
println("=========================================")


println("\n--- Тест DFA (ИСПРАВЛЕН): Язык a*b ---")
dfa_states = "1, 2, 3"
dfa_start = "1"
dfa_final = "2"

dfa_transitions = "1,a,1; 1,b,2; 2,a,3; 2,b,3; 3,a,3; 3,b,3"

dfa_ab = parse_dfa(dfa_states, dfa_start, dfa_final, dfa_transitions)

println("DFA: 'aaab' (НЕ a*b) принимается: ", check_dfa(dfa_ab, "aaab"))  # false
println("DFA: 'b' (a*b) принимается: ", check_dfa(dfa_ab, "b"))        # true
println("DFA: 'ab' (a*b) принимается: ", check_dfa(dfa_ab, "ab"))      # true
println("DFA: 'aaabb' (НЕ a*b) принимается: ", check_dfa(dfa_ab, "aaabb")) # false



println("\n--- Тест NFA (ИСПРАВЛЕН): Язык (a|b)*b с ε-переходом ---")
nfa_states = "1, 2"
nfa_start = "1"
nfa_final = "2"

nfa_transitions = "1,a,1; 1,b,1,2; 2,ε,1"

nfa_star_b = parse_nfa(nfa_states, nfa_start, nfa_final, nfa_transitions)

println("NFA: 'bab' (оканчивается на b) принимается: ", check_nfa(nfa_star_b, "bab"))    # true
println("NFA: 'aabba' (не оканчивается на b) принимается: ", check_nfa(nfa_star_b, "aabba")) # false
println("NFA: 'b' (оканчивается на b) принимается: ", check_nfa(nfa_star_b, "b"))        # true

       ДЕМОНСТРАЦИЯ (DFA И NFA)          

--- Тест DFA (ИСПРАВЛЕН): Язык a*b ---
DFA: 'aaab' (НЕ a*b) принимается: true
DFA: 'b' (a*b) принимается: true
DFA: 'ab' (a*b) принимается: true
DFA: 'aaabb' (НЕ a*b) принимается: false

--- Тест NFA (ИСПРАВЛЕН): Язык (a|b)*b с ε-переходом ---
NFA: 'bab' (оканчивается на b) принимается: true
NFA: 'aabba' (не оканчивается на b) принимается: false
NFA: 'b' (оканчивается на b) принимается: true


In [3]:

struct CFA{T}
    states::Set{T}
    start_state::T
    final_states::Set{T}
    
    and_states::Set{T}
    
    transitions::Dict{Tuple{T, Union{Char, Nothing}}, Set{T}}
end


function parse_states(s::AbstractString)::Set{Int}
    if isempty(strip(s)) return Set{Int}() end
    return Set(parse(Int, strip(x)) for x in split(s, ','))
end

function parse_cfa(states_str, start_str, final_str, and_str, transitions_str)::CFA{Int}
    states = parse_states(states_str)
    start_state = parse(Int, strip(start_str))
    final_states = parse_states(final_str)
    and_states = parse_states(and_str) 
    transitions = Dict{Tuple{Int, Union{Char, Nothing}}, Set{Int}}()

    for entry in split(transitions_str, ';', keepempty=false)
        parts = split(entry, ',', limit=3)
        from = parse(Int, strip(parts[1]))
        sym_str = strip(parts[2])
        to_list = parse_states(parts[3])

        sym_key = (sym_str == "ε" || sym_str == "E") ? nothing : first(sym_str)
        key = (from, sym_key)

        if haskey(transitions, key)
            union!(transitions[key], to_list)
        else
            transitions[key] = to_list
        end
    end
    return CFA(states, start_state, final_states, and_states, transitions)
end




function check_cfa(cfa::CFA{Int}, word::AbstractString)::Bool
    
    memo = Dict{Tuple{Int, Int}, Bool}()

    
    path_visited = Set{Tuple{Int, Int}}()

    
    function solve(state::Int, idx::Int)::Bool
        
        memo_key = (state, idx)
        if haskey(memo, memo_key)
            return memo[memo_key]
        end

        
        if memo_key in path_visited
            return false 
        end
        push!(path_visited, memo_key)

        is_end = idx > length(word)
        current_char = is_end ? nothing : word[idx]

        
        is_accepted_here = is_end && (state in cfa.final_states)

        
        results = Vector{Bool}()

        
        if !is_end
            key_char = (state, current_char)
            if haskey(cfa.transitions, key_char)
                for next_st in cfa.transitions[key_char]
                    push!(results, solve(next_st, idx + 1))
                end
            end
        end

        
        key_eps = (state, nothing)
        if haskey(cfa.transitions, key_eps)
            for next_st in cfa.transitions[key_eps]
                push!(results, solve(next_st, idx)) 
            end
        end

        
        is_and_node = state in cfa.and_states
        final_res = false

        if is_and_node
            
            if isempty(results)
                final_res = is_accepted_here
            else
                
                final_res = all(results)
            end
        else
            
            if isempty(results)
                final_res = is_accepted_here
            else
                final_res = is_accepted_here || any(results)
            end
        end

        pop!(path_visited, memo_key)
        memo[memo_key] = final_res
        return final_res
    end

    return solve(cfa.start_state, 1)
end

check_cfa (generic function with 1 method)

In [4]:

nfa_states = "1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13"


nfa_start = "1"


nfa_final = "11"


nfa_transitions = "1,a,2,9; 1,b,9; 2,b,3; 2,a,4; 3,a,5; 5,a,6; 6,a,7; 7,a,1; 4,b,6; 4,a,8; 8,b,7; 9,a,10; 9,a,10; 10,a,11; 10,b,11; 11,a,12; 12,a,13; 12,b,11; 13,b,11"

nfa_complex = parse_nfa(nfa_states, nfa_start, nfa_final, nfa_transitions)


dfa_states = "0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18"


dfa_start = "5"


dfa_final = "0, 1, 2, 3, 4"


dfa_transitions = "5,a,6; 5,b,14; 6,a,15; 6,b,12; 14,a,17; 15,a,0; 15,b,1; 12,a,11; 17,a,4; 17,b,4; 0,a,13; 0,b,8; 1,a,9; 11,a,10; 4,a,13; 13,a,18; 13,b,4; 8,a,5; 9,a,7; 9,b,4; 10,a,8; 18,b,4; 7,a,6; 7,b,2; 2,a,16; 16,a,3; 16,b,4; 3,a,13; 3,b,4"

dfa_complex  = parse_dfa(dfa_states, dfa_start, dfa_final, dfa_transitions)


DFA{Int64}(Set([5, 16, 7, 12, 8, 17, 1, 0, 4, 6, 13, 2, 10, 11, 9, 15, 18, 14, 3]), 5, Set([0, 4, 2, 3, 1]), Dict((7, 'a') => 6, (17, 'b') => 4, (0, 'b') => 8, (6, 'b') => 12, (9, 'b') => 4, (4, 'a') => 13, (15, 'a') => 0, (13, 'a') => 18, (2, 'a') => 16, (10, 'a') => 8…))

In [5]:

struct CFA{T}
    states::Set{T}
    start_state::T
    final_states::Set{T}
    
    and_states::Set{T}
    transitions::Dict{Tuple{T, Union{Char, Nothing}}, Set{T}}
end

function parse_states(s::AbstractString)::Set{Int}
    if isempty(strip(s)) return Set{Int}() end
    return Set(parse(Int, strip(x)) for x in split(s, ','))
end

function parse_cfa(states_str, start_str, final_str, and_str, transitions_str)::CFA{Int}
    states = parse_states(states_str)
    start_state = parse(Int, strip(start_str))
    final_states = parse_states(final_str)
    and_states = parse_states(and_str) 
    transitions = Dict{Tuple{Int, Union{Char, Nothing}}, Set{Int}}()

    for entry in split(transitions_str, ';', keepempty=false)
        parts = split(entry, ',', limit=3)
        from = parse(Int, strip(parts[1]))
        sym_str = strip(parts[2])
        to_list = parse_states(parts[3])

        sym_key = (sym_str == "ε" || sym_str == "E") ? nothing : first(sym_str)
        key = (from, sym_key)

        if haskey(transitions, key)
            union!(transitions[key], to_list)
        else
            transitions[key] = to_list
        end
    end
    return CFA(states, start_state, final_states, and_states, transitions)
end




function check_cfa(cfa::CFA{Int}, word::AbstractString)::Bool
    
    memo = Dict{Tuple{Int, Int}, Bool}()

    path_visited = Set{Tuple{Int, Int}}()

    function solve(state::Int, idx::Int)::Bool
        
        memo_key = (state, idx)
        if haskey(memo, memo_key)
            return memo[memo_key]
        end

        
        if memo_key in path_visited
            return false 
        end
        push!(path_visited, memo_key)

        is_end = idx > length(word)
        current_char = is_end ? nothing : word[idx]

        
        is_accepted_here = is_end && (state in cfa.final_states)

        
        results = Vector{Bool}()

        
        if !is_end
            key_char = (state, current_char)
            if haskey(cfa.transitions, key_char)
                for next_st in cfa.transitions[key_char]
                    push!(results, solve(next_st, idx + 1))
                end
            end
        end

        
        key_eps = (state, nothing)
        if haskey(cfa.transitions, key_eps)
            for next_st in cfa.transitions[key_eps]
                push!(results, solve(next_st, idx)) 
            end
        end

        
        is_and_node = state in cfa.and_states
        final_res = false

        if is_and_node
            
            if isempty(results)
                final_res = is_accepted_here
            else
                
                final_res = all(results)
            end
        else
            
            if isempty(results)
                final_res = is_accepted_here
            else
                final_res = is_accepted_here || any(results)
            end
        end

        pop!(path_visited, memo_key)
        memo[memo_key] = final_res
        return final_res
    end

    return solve(cfa.start_state, 1)
end


check_cfa (generic function with 1 method)

In [6]:
println("=========================================")
println("       ВВОД НОВОГО CFA (DIGRAPH)         ")
println("=========================================")


cfa_states = "1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21"


cfa_start = "1"



cfa_final = "11, 17, 19,20"


cfa_and = "1"


cfa_transitions = join([
    
    "1,ε,21",
    "21,a,2",      
    "21,a,9",      
    "21,b,9",      

    
    "2,b,3",
    "2,a,4",
    "3,a,5",
    "5,a,6",
    "6,a,7",
    "7,a,14",
    "14,a,2",
    "14,a,15",     
    "14,b,15",
    "15,a,16",
    "16,a,17",     
    "16,b,17",
    "17,a,17",     
    "17,b,17",     

    
    "4,b,6",
    "4,a,8",
    "8,b,7",

    
    "1,ε,18",
    "18,a,18",     
    "18,b,18",     
    "18,a,9",      
    "18,b,9",      

    
    "9,a,10",
    "10,a,11",     
    "10,b,11",     
    "11,a,12",
    "12,a,13",
    "13,b,11",
    "12,b,11",

    
    "1,ε,19",
    "19,a,19",     
    "19,b,20",     
    "20,a,19"      
], "; ")


cfa_graph = parse_cfa(cfa_states, cfa_start, cfa_final, cfa_and, cfa_transitions)

println("CFA успешно загружен.")



println("Тест 'aaa': ", check_cfa(cfa_graph, "aaa"))


       ВВОД НОВОГО CFA (DIGRAPH)         
CFA успешно загружен.
Тест 'aaa': true


In [7]:
using Random

abstract type Recognizer end

struct RegexWrapper <: Recognizer
    re::Regex
end
check(r::RegexWrapper, w) = is_word_in_language(r.re, w) 
name(r::RegexWrapper) = "Regex"

struct DFAWrapper <: Recognizer
    model::DFA
end
check(d::DFAWrapper, w) = check_dfa(d.model, w)
name(d::DFAWrapper) = "DFA"

struct NFAWrapper <: Recognizer
    model::NFA
end
check(n::NFAWrapper, w) = check_nfa(n.model, w)
name(n::NFAWrapper) = "NFA"

struct CFAWrapper <: Recognizer
    model::CFA
end
check(c::CFAWrapper, w) = check_cfa(c.model, w)
name(c::CFAWrapper) = "CFA (PKA)"


function generate_random_words(alphabet::Vector{Char}, count::Int, max_len::Int)
    words = String[]
    push!(words, "") 
    for _ in 1:count
        len = rand(1:max_len)
        w = join(rand(alphabet, len))
        push!(words, w)
    end
    return unique(words)
end


function generate_accepted_bfs(automaton, max_count::Int, max_len::Int)
   

    valid_words = String[]
    
    visited = Set{Tuple{Int, Int}}()

    
    queue = Vector{Tuple{Int, String}}()
    push!(queue, (automaton.start_state, ""))

    alphabet = ['a', 'b'] 

    while !isempty(queue) && length(valid_words) < max_count
        state, word = popfirst!(queue)

        
        if length(word) > max_len
            continue
        end

        
        if state in automaton.final_states
            push!(valid_words, word)
        end

        
        for char in alphabet
            next_states = Int[]

            
            if hasfield(typeof(automaton), :transitions)
                
                if isa(automaton, DFA)
                    key = (state, char)
                    if haskey(automaton.transitions, key)
                        push!(next_states, automaton.transitions[key])
                    end
                
                elseif isa(automaton, NFA)
                    key = (state, char)
                    if haskey(automaton.transitions, key)
                        
                        append!(next_states, collect(automaton.transitions[key]))
                    end
                    
                end
            end

            for next_st in next_states
                if (next_st, length(word) + 1) ∉ visited
                    push!(visited, (next_st, length(word) + 1))
                    push!(queue, (next_st, word * char))
                end
            end
        end
    end

    return unique(valid_words)
end

function verify_equivalence(models::Vector{Recognizer})
    println("\n🔍 НАЧАЛО АВТОМАТИЧЕСКОГО ТЕСТИРОВАНИЯ ЭКВИВАЛЕНТНОСТИ")
    println("Модели в тесте: ", join([name(m) for m in models], ", "))

    errors_found = 0

    
    println("\n--- [Фаза 1] Случайные слова (Random Fuzzing) ---")
    random_words = generate_random_words(['a', 'b'], 100, 20) 

    all_words_to_test = copy(random_words)

    
    println("--- [Фаза 2] Генерация 'правильных' слов из структур автоматов ---")
    for m in models
        if isa(m, DFAWrapper) || isa(m, NFAWrapper)
            println("Генерация примеров из $(name(m))...")
            
            valid_samples = generate_accepted_bfs(m.model, 10, 10)
            append!(all_words_to_test, valid_samples)
        end
    end

    unique!(all_words_to_test)
    println("Всего уникальных тестов: $(length(all_words_to_test))")

    
    println("\n--- [Фаза 3] Сверка результатов ---")

    
    header = rpad("WORD", 12) * " | " * join([rpad(name(m), 8) for m in models], " | ") * " | STATUS"
    println("-" ^ length(header))
    println(header)
    println("-" ^ length(header))

    for word in all_words_to_test
        results = Bool[]
        for model in models
            push!(results, check(model, word))
        end

        
        is_consistent = all(r -> r == results[1], results)

        status = is_consistent ? "✅ OK" : "❌ MISMATCH"
        if !is_consistent
            errors_found += 1
        end

        
        row = rpad(word == "" ? "ε" : word, 12) * " | " * join([rpad(string(r), 8) for r in results], " | ") * " | " * status
        println(row)
    end

    println("-" ^ length(header))
    if errors_found == 0
        println("\n УСПЕХ: Все распознаватели эквивалентны на $(length(all_words_to_test)) тестах.")
    else
        println("\n НАЙДЕНО ОШИБОК: $errors_found. Распознаватели НЕ эквивалентны.")
    end
end

verify_equivalence (generic function with 1 method)

In [8]:
academic_regex = r"(aabaa|aaaba|abaaaa)*(a|b)a(a|b)(ab|aab)*"

wrappers = [
    RegexWrapper(academic_regex),
    DFAWrapper(dfa_complex), 
    NFAWrapper(nfa_complex), 
    CFAWrapper(cfa_graph)  
]


verify_equivalence(wrappers)


🔍 НАЧАЛО АВТОМАТИЧЕСКОГО ТЕСТИРОВАНИЯ ЭКВИВАЛЕНТНОСТИ
Модели в тесте: Regex, DFA, NFA, CFA (PKA)

--- [Фаза 1] Случайные слова (Random Fuzzing) ---
--- [Фаза 2] Генерация 'правильных' слов из структур автоматов ---
Генерация примеров из DFA...
Генерация примеров из NFA...
Всего уникальных тестов: 104

--- [Фаза 3] Сверка результатов ---
------------------------------------------------------------------
WORD         | Regex    | DFA      | NFA      | CFA (PKA) | STATUS
------------------------------------------------------------------
ε            | false    | false    | false    | false    | ✅ OK
aababbbbbbbbbabaaabb | false    | false    | false    | false    | ✅ OK
bbaa         | false    | false    | false    | false    | ✅ OK
bbba         | false    | false    | false    | false    | ✅ OK
baabbaabbbabbbbbb | false    | false    | false    | false    | ✅ OK
bab          | true     | true     | true     | true     | ✅ OK
aabbbbbbaaabaaaa | false    | false    | false    | false    |